## 1st Step. Create a Notebook for the Assigment

# My Peer-Graded Assigment: Segmenting and Clustering Neighborhoods in Toronto

First Part is to install the proper libraries

In [1]:
#Installin Beautiful Soup
import sys
!{sys.executable} -m pip install bs4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import codecs

## 2nd Step. Get the information form Wikipedia

Second Part of the assigment is to create the dataframe in pandas of the neighborhood of Toronto

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    'User-Agent': 'Mozilla/5.0'
} # Needed to prevent 403 error on Wikipedia
page = requests.get(wiki, headers=header)
soup = BeautifulSoup(page.content)

tables = soup.findAll("table", {"class": "wikitable"})

# show tables
for table in tables:
    print("###############")
    print(table.text[:100])

for tn in range(len(tables)):
    table = tables[tn]

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    data[i + k][j + l + m] += cell.text

        data.append(rowD)

    # write data out to tab seperated format
    page = os.path.split(wiki)[1]
    fname = 'output_{}_t{}.csv'.format(page, tn)
    f = codecs.open(fname, 'w')
    for i in range(nrows):
        rowStr = '\t'.join(data[i])
        rowStr = rowStr.replace('\n', '')
        print(rowStr)
        f.write(rowStr + '\n')
    f.close()

###############


Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M
Postcode	Borough	Neighbourhood
M1A	Not assigned	Not assigned
M2A	Not assigned	Not assigned
M3A	North York	Parkwoods
M4A	North York	Victoria Village
M5A	Downtown Toronto	Harbourfront
M5A	Downtown Toronto	Regent Park
M6A	North York	Lawrence Heights
M6A	North York	Lawrence Manor
M7A	Queen's Park	Not assigned
M8A	Not assigned	Not assigned
M9A	Etobicoke	Islington Avenue
M1B	Scarborough	Rouge
M1B	Scarborough	Malvern
M2B	Not assigned	Not assigned
M3B	North York	Don Mills North
M4B	East York	Woodbine Gardens
M4B	East York	Parkview Hill
M5B	Downtown Toronto	Ryerson
M5B	Downtown Toronto	Garden District
M6B	North York	Glencairn
M7B	Not assigned	Not assigned
M8B	Not assigned	Not assigned
M9B	Etobicoke	Cloverdale
M9B	Etobicoke	Islington
M9B	Etobicoke	Martin Grove
M9B	Etobicoke	Princess Gardens
M9B	Etobicoke	West Deane Park
M1C	Scarborough	Highland Creek
M1C	Scarborough	Rouge Hill
M1C

## 3rd Step. Create the information in Pandas Dataframe

First we put the info in a data frame and clean up the data as requested

In [4]:
df = pd.read_csv('output_List_of_postal_codes_of_Canada:_M_t0.csv', sep='\t')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df.shape

(288, 3)

 ### *Ignore rows with Not assigned*

In [6]:
# Show the first 15 rows of the original dataframe and the shape to compare
print("Given Dataframe :\n", df.head(15))  
print("Shape of Given Dataframe", df.shape)

# Filter rows that does not content "Not assigned"
options = ['Not assigned']
df_tor = df.loc[~df['Borough'].isin(options)] 

# Show the first 15 rows of the resultant dataframe and shape
print('\nResult Dataframe :\n', df_tor.head(15)) 
print("Shape of Result Dataframe", df_tor.shape)

Given Dataframe :
    Postcode           Borough     Neighbourhood
0       M1A      Not assigned      Not assigned
1       M2A      Not assigned      Not assigned
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  Downtown Toronto      Harbourfront
5       M5A  Downtown Toronto       Regent Park
6       M6A        North York  Lawrence Heights
7       M6A        North York    Lawrence Manor
8       M7A      Queen's Park      Not assigned
9       M8A      Not assigned      Not assigned
10      M9A         Etobicoke  Islington Avenue
11      M1B       Scarborough             Rouge
12      M1B       Scarborough           Malvern
13      M2B      Not assigned      Not assigned
14      M3B        North York   Don Mills North
Shape of Given Dataframe (288, 3)

Result Dataframe :
    Postcode           Borough     Neighbourhood
2       M3A        North York         Parkwoods
3       M4A        North York  Victoria Village
4       M5A  D

 ### *Combine Rows with same Postcode adding Neighbourhood names with commas*

In [8]:
#Combine duplicated rows by groupby
df_tor1 = (df_tor.groupby(['Postcode','Borough'])['Neighbourhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

# Show the first 15 rows of the dataframe and shap
print('Dataframe :\n', df_tor1.head(15)) 
print("Shape of Dataframe", df_tor1.shape)

Dataframe :
    Postcode      Borough                                      Neighbourhood
0       M1B  Scarborough                                      Malvern,Rouge
1       M1C  Scarborough               Highland Creek,Rouge Hill,Port Union
2       M1E  Scarborough                    Morningside,Guildwood,West Hill
3       M1G  Scarborough                                             Woburn
4       M1H  Scarborough                                          Cedarbrae
5       M1J  Scarborough                                Scarborough Village
6       M1K  Scarborough          Ionview,Kennedy Park,East Birchmount Park
7       M1L  Scarborough                      Golden Mile,Clairlea,Oakridge
8       M1M  Scarborough      Cliffside,Scarborough Village West,Cliffcrest
9       M1N  Scarborough                         Cliffside West,Birch Cliff
10      M1P  Scarborough  Scarborough Town Centre,Wexford Heights,Dorset...
11      M1R  Scarborough                                   Wexford,Maryvale

 ### *Assign Neighborhood name in the Not Assigned Cells*

In [9]:
df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Neighbourhood']=df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Borough']
print('Dataframe :\n', df_tor1.tail(20)) 
print("Shape of Dataframe", df_tor1.shape)

Dataframe :
     Postcode       Borough                                      Neighbourhood
83       M6R  West Toronto                              Parkdale,Roncesvalles
84       M6S  West Toronto                                  Swansea,Runnymede
85       M7A  Queen's Park                                       Queen's Park
86       M7R   Mississauga              Canada Post Gateway Processing Centre
87       M7Y  East Toronto  Business Reply Mail Processing Centre 969 Eastern
88       M8V     Etobicoke         Mimico South,New Toronto,Humber Bay Shores
89       M8W     Etobicoke                              Alderwood,Long Branch
90       M8X     Etobicoke        The Kingsway,Montgomery Road,Old Mill North
91       M8Y     Etobicoke  Sunnylea,Kingsway Park South East,Mimico NE,Th...
92       M8Z     Etobicoke  Kingsway Park South West,The Queensway West,Mi...
93       M9A     Etobicoke                                   Islington Avenue
94       M9B     Etobicoke  Princess Gardens,West D

## Step 4. Put the file in my repostery Github

Thanks